### Müllabfuhr - Wahl von 5 Pfaden

In [34]:
%%time
'''
6: 190 ms, 7: 22.6 s, 8: 3min 3s     
'''
inf = float('inf')
nr = 2
f = open('./beispieldaten/muellabfuhr'+str(nr)+'.txt')
anzV, anzE = [int(x) for x in f.readline().split()]

G = [[inf]*anzV for _ in range(anzV)]
V = set()
E = set()
for i in range(anzE):
    von, bis, km = [int(x) for x in f.readline().split()]
    G[von][bis]=km
    G[bis][von]=km
    V.add(von)
    V.add(bis)
    
    if von < bis:
        E.add((von,bis))
    else:
        E.add((bis,von))
for i in range(anzV):
    G[i][i]=0
    
def floyd(c):
    n = len(c)
    d = [[0]*n for j in range(n)]   # Matrix D
    p = [[0]*n for j in range(n)]   # Matrix P
    for i in range(n):
        for j in range(n):
            d[i][j] = c[i][j]
            p[i][j] = i

    for k in range(n):
        for i in range(n):
            for j in range(n):
                tmp = d[i][k] + d[k][j]
                if tmp < d[i][j]:
                    d[i][j] = tmp
                    p[i][j] = p[k][j]
    return d, p

def getPath(p, i, j):
    
    if i == j: return (i,)
    return getPath(p,i,p[i][j]) + (j,)

def printMatrix(a):
    for i in range(len(a)):
        for j in range(len(a)):
            if a[i][j] == inf:
                print("{:4}".format("."),end='')
            else:
                print("{:<4}".format(a[i][j]),end='')
        print()
    print()
    
D, P = floyd(G)


Wall time: 3.45 ms


In [35]:
def pathDist(p,k):
    a,b = k
    v = p[-1]
    return pathCost(p)+min(D[v][a],D[v][b])

def zurKante(p,k):
    ''' kürzeste Pfad vom Pfad p zur Kante k '''
    a, b = k
    v = p[-1]
    p0 = list(p)[:-1]
    ziel = a
    if D[v][a] < D[v][b]:
        return getPath(P,v,a)+(b,)
    else:
        return tuple(p0)+getPath(P,v,b)+(a,)
        
def pathKanten(p):
    ''' set mit den normierten Kanten von p '''
    tmp = set()
    for i in range(len(p)-1):
        a, b = p[i], p[i+1]
        if a < b: tmp.add((a,b))
        else: tmp.add((b,a))
    return tmp   

def pathCost(p):
    tmp = 0 
    for i in range(len(p)-1):
        a, b = p[i], p[i+1]
        tmp+=G[a][b]
    return tmp

def cost(pp):    
    max_cost = 0
    for p in pp:
        if pathCost(p)> max_cost:
            max_cost = pathCost(p)
    return max_cost


In [36]:
pp = [(0,),(0,),(0,),(0,),(0,)]
E1 = E              # die noch zu überdeckenden Kanten
 

while len(E1)>0:
    gewichte = {(a,b): D[0][a]+G[a][b]+D[b][0] for a,b in E1}
    reihenfolge = sorted(E1,key = lambda x : -gewichte[x])
    kante  = reihenfolge[0]
    dd = [pathDist(p,kante) for p in pp]
    i = dd.index(min(dd))  
    path = pp[i]
    
    p1 = zurKante(path,kante)
    pp[i] = p1
    E1 = E1 - pathKanten(p1)

# jetzt auf dem kürzesten Weg zurück
for i in range(5):
    p = pp[i]
    v = p[-1]
    p1 = tuple(list(p[:-1]))
    pp[i] = p1 + getPath(P,v,0)
    
for p in pp:
    print(pathCost(p),p)
print("Maximaler Weg:",cost(pp))

9 (0, 9, 7, 8, 2, 8, 7, 1, 6, 0)
8 (11, 2, 14, 10, 9, 12, 1, 6, 0)
4 (14, 5, 9, 6, 0)
7 (0, 9, 10, 2, 10, 4, 6, 0)
8 (3, 13, 14, 13, 4, 13, 1, 6, 0)
Maximaler Weg: 9


-----

-------